<a href="https://colab.research.google.com/github/5neaX/chessgame/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
# Installing dependencies
!pip install python-chess

In [106]:
import chess
import random
import math
from math import inf 
from chess import Board, Move

from P6_Assignment2_Notebook_PythonChess_Agent import ChessAgent
from P6_Assignment2_Notebook_PythonChess_Env import play_game, zobrist_hash




In [107]:
# Openings to investigate
# CAUTION: Will take time on higher depths ( > 3)

ClosedSicilianDefence = 'rnbqkbnr/pp1ppppp/8/2p5/4P3/2N5/PPPP1PPP/R1BQKBNR b KQkq - 1 2'
ViennaOpenning = 'rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQKBNR b KQkq - 1 2'
NimzoIndianDefence = 'rnbqk2r/pppp1ppp/4pn2/8/1bPP4/2N5/PP2PPPP/R1BQKBNR w KQkq - 2 4'
SlavDefence = 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3'
QueenzGambit = 'rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBNR b KQkq c3 0 2'
RuyLopez = 'r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3'

In [108]:
# Simple End Games - White Wins
WhiteQueenMate_M1 = '7k/8/6K1/6Q1/8/8/8/8 w - - 0 1'
WhiteKingRookMate_M1 = '7k/8/6K1/8/8/8/8/5R2 w - - 0 1'
WhiteTwoRooksMate_M2 = 'K5k1/8/8/8/8/8/5R2/4R3 w - - 0 1'
WhiteKingRookMate_M2 = '6k1/8/5RK1/8/8/8/8/8 w - - 0 1'

# Simple End Games - Black Wins
BlackQueenMate_M1 = '8/8/8/8/5q2/6k1/8/7K b - - 0 1'
BlackKingRookMate_M1 = '8/8/8/8/8/5rk1/8/7K b - - 0 1'
BlackTwoRooksMate_M2 = '8/K5k1/8/8/8/8/3r4/2r5 b - - 0 1'
BlackKingRookMate_M1 = '8/8/8/8/8/5rk1/8/7K b - - 0 1'

# Deeper End Games - Black Wins
BlackTwoRooksMate_M2 = 'rrk5/8/8/8/8/8/8/4K3 b - - 0 1'
BlackQueenRookMate_M2 = 'qr1k4/8/8/8/8/8/8/4K3 b - - 0 1'
BlackRookKingMate_M13 = 'r2k4/8/8/8/8/8/8/4K3 b - - 0 1'
BlackQueenKingMate_M8 = 'q2k4/8/8/8/8/8/8/4K3 b - - 0 1'

# Deeper End Games - White Wins
WhiteTwoRooksMate_M3 = '3k4/8/8/8/8/8/8/4KRR1 w - - 0 1'
WhiteQueenRookMate_M2 = '3k4/8/8/8/8/8/8/4KRQ1 w - - 0 1'
WhiteRookKingMate_M20 = '3k4/8/8/8/8/8/8/4KR2 w - - 0 1'
WhiteQueenKingMate_M7 = '3k4/8/8/8/8/8/8/4K1Q1 w - - 0 1'

# End Games - Forcing Draw
WhitePromote2ForceDraw = '8/3P1k2/2q5/4K3/8/8/8/8 w - - 0 1'
BlackPromote2ForceDraw = '8/8/8/2k5/8/3K4/4p1Q1/8 b - - 0 1'

# Early Mate - White Wins
WhiteEarlyGame_M1 = 'rnbqkbnr/ppppp2p/8/5pp1/4P2P/8/PPPP1PP1/RNBQKBNR w KQkq - 0 1'
WhiteMidgame_h5SmutherMate_M5 = '2nrkbn1/pp1bp3/2qprp1p/6p1/4P2P/2NP1N2/PPP2PP1/R1BQKB1R w KQ - 0 1'

# White must NOT take on f7 using the Knight. Instead it should take with the light-square Bishop
WhiteAntiTraxler = 'r1bqk2r/pppp1ppp/2n2n2/2b1p1N1/2B1P3/8/PPPP1PPP/RNBQK2R w KQkq - 0 1'

# Black must ignore the fork by the white knight on f7 (forking queen and rook) and 
# insted sacrifice the dark-square bishop by taking on f2 with a check and chase down 
# the white king
BlackTraxler = 'r1bqk2r/pppp1Npp/2n2n2/2b1p3/2B1P3/8/PPPP1PPP/RNBQK2R b KQkq - 0 1'

# Courtsey of International Master Eric Rosen
# Black must ignore the mate thread of the pawn on e7 (e.g. white to capture the 
# black queen deliver mate). Instead, black must sacrifice dark-square bishop on 
# c5 by taking the white pawn on f2 with a check. The white king must take the 
# bishop (only move). Then black queen can take the white queen for free.
IMROSEN = 'rnbqk2r/ppp1Pppp/8/2b5/8/5N2/PPP1PPPP/RNBQKB1R b KQkq - 0 1'

# Black to sacrifice the queen on d2. White to refuse 
WhiteBackrankTemptation = '1k2r3/ppp5/3p4/6q1/8/8/5PPP/3R2K1 b - - 0 1'
BlackBackrankDecision = '1k2r3/ppp5/3p4/8/3q4/8/5PPP/3R2K1 w - - 0 1'

WhiteQueenSacMate2 = 'r1bqk2r/ppppnp1p/2n3p1/3N2B1/2Pb4/8/PP2PPPP/R2QKB1R w KQkq - 0 1'

# White must sacrifice the queen (only move) to prevent mate. 
WhiteMidgamePreventM1 = 'r1b1kbnr/pppp1Npp/8/8/2Bnq3/8/PPPP1P1P/RNBQKR2 w Qkq - 0 1'

# Same position where white spared the queen and blocked with the bishop. 
# Black must deliver mate with Nf3
BlackMidgameMate_M1 = 'r1b1kbnr/pppp1Npp/8/8/3nq3/8/PPPPBP1P/RNBQKR2 b Qkq - 0 1'



In [109]:
# Sample scoring function

def isDraw(board):
    return (board.is_fivefold_repetition() or
            board.is_seventyfive_moves() or 
            board.is_insufficient_material() or
            board.is_stalemate() or
            board.can_claim_draw() or 
            board.can_claim_fifty_moves() or
            board.can_claim_threefold_repetition())

def drawScore(board):
    return not isDraw(board)

def mateScore(board):
    return 1000 if board.is_checkmate() else 0

def boardScore(board, color):
    score = 0
    for piece, value in [(chess.ROOK, 5), 
                        (chess.QUEEN, 9), 
                        (chess.BISHOP, 3), 
                        (chess.KNIGHT, 3), 
                        (chess.PAWN, 1), 
                        (chess.KING, 0)]:
        score += len(board.pieces(piece, color)) * value
        score -= len(board.pieces(piece, not color)) * value
        pass
    
    score += mateScore(board)   # Check mate score
    score *= drawScore(board)   # Neutralise for draw
    
    return score


In [110]:
# Quick Helper Functions
def newBoard():
    return chess.Board()

def resetBoard(b):
    return b.reset()

def showBoard(b):
    # Only shows a single instance. Cannot be used in a loop.
    # To visualise the game use playGame(whiteAgent, blackAgent, ...)
    return chess.svg.board(b)

def randomMove(b):
    return random.choice(list(b.legal_moves))

def legalMoves(b):
    return list(b.legal_moves)

def makeMove(b, mv):
    return b.push(mv)
    
def takeBack(b):
    return b.pop(mv)

def zobristHash(b):
    return zobrist_hash(b)
    
def playGame(whiteAgent, blackAgent, 
             fen='rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 
             scoreFunc=boardScore,
             dtime=.01):
    return play_game(whiteAgent, blackAgent, 
                     fen=fen,
                     pause=dtime)

In [111]:
class RandomChessAgent(ChessAgent):
    def getAction(self, board):
        # get a move object
        move = random.choice(list(board.legal_moves))
        
        # convert move object to UCI standard (d2d4 --> move d2 pawn to d4)
        moveUCI = move.uci()
        
        # return move
        return moveUCI
    

In [182]:
class MinMaxChessAgent(ChessAgent):
    def __init__(self,chessagent,depth=3,verbose=False):
        super().__init__(chessagent, "MinMaxAlphaBetaSearch")
        self.depth=depth
        self.verbose=verbose 
        # e.g. self.transpositionTable = dict()
        pass
    
    def score(self, board):
        score = 0
        
        # Your code goes here ...
        # Check the sample score function for ideas ...
        # Consult the chess-python docs at https://python-chess.readthedocs.io/en/latest/
        
        return score
    
    def _MinMaxSearch(self, board,chessagent, depth=3):
        bestAction = random.choice(list(board.legal_moves))
        
       
      
        return bestAction
    
    def _MinMaxAlphaBetaSearch(self, board,chessagent, alpha=-inf, beta=inf, depth =3):
        if depth==0:
           return [board, self.chessagent,boardScore, None]
        bestAction = random.choice(list(board.legal_moves))
        
        
        
        return bestAction
    
    def MinMaxAlphaBetaTranspositionSearch(self, board, 
                                           # ... other search-specific parameters
                                           alpha=-inf, beta=inf,
                                           depth=3):
        bestAction = random.choice(list(board.legal_moves))
        
        return bestAction
    
    def getAction(self, board):
        # get a move object
        move = self._MinMaxAlphaBetaSearch(board, ChessAgent)
        
        # convert move object to UCI standard (d2d4 --> move d2 pawn to d4)
        moveUCI = move.uci()
        
        # return move
        return moveUCI

In [183]:
playGame(whiteAgent=MinMaxChessAgent(ChessAgent), 
         blackAgent=RandomChessAgent(ChessAgent), 
         # fen=BlackTraxler,
         # fen='rnbqk2r/ppp1Pppp/8/2b5/8/5N2/PPP1PPPP/RNBQKB1R b KQkq - 0 1',
         # fen=WhiteTwoRooksMate_M3,
         dtime=.05)

draw: insufficient material


(None,
 'draw: insufficient material',
 Board('2N5/1K6/8/2k5/8/8/8/8 b - - 0 171'),
 0)